In [2]:
!pip install s2cloudless

  Obtaining dependency information for s2cloudless from https://files.pythonhosted.org/packages/72/53/a535d72eb57667f8e408013c85fce518516092a63887040888076cf32a07/s2cloudless-1.7.1-py3-none-any.whl.metadata
  Using cached s2cloudless-1.7.1-py3-none-any.whl.metadata (26 kB)
  Using cached lightgbm-4.1.0.tar.gz (1.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for opencv-python-headless from https://files.pythonhosted.org/packages/12/0f/b87324db284c54d1d1a1c1242a128fb18515915d124325784c90f23d8ef5/opencv_python_headless-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata
  Using cached opencv_python_headless-4.8.1.78-cp37-abi3-macosx_11_0_arm64.whl.metadata (19 kB)
  Obtaining dependency information for sentinelhub>=3.9.0 from https://files.pythonhosted.org/packages/b0/62/7520804a7ea9f5f5a23784bbf7f5299bca37aba2055c52d461

In [1]:
import tensorflow as tf
import tensorflow_hub as hub


In [2]:
import glob
import random
import numpy as np

In [3]:
import os

In [4]:
import tifffile

In [5]:
def get_lulc_class(path):
  splits = path.split('/')
  return splits[-2]

In [62]:
def multispectral_to_rgb(raster, optical_maximum = 2000):

  r = raster[:, :, 3]
  g = raster[:, :, 2]
  b = raster[:, :, 1]

  rgb_raster = np.stack([r, g, b], axis=2)
  rgb_raster = np.nan_to_num(rgb_raster)
  rgb_raster = np.log(rgb_raster)
  min = np.percentile(rgb_raster.flatten(), 1)
  max = np.percentile(rgb_raster.flatten(), 99)
  rgb_raster = 255 * (rgb_raster - min)/(max-min)
  
  #rgb_raster = rgb_raster/np.nanmax(raster)
  #rgb_raster = np.around(rgb_raster*255)
  rgb_raster = np.nan_to_num(np.clip(rgb_raster, 0, 255)).astype(int)
  return rgb_raster

In [61]:
classDictionary = {30: 'Herbaceous Vegetation',
20: 'Shrubs',
40: 'Agricultural Land',
50: 'Urban Areas',
60: 'Bare Earth and Sparse Vegetation',
70: 'Snow and Ice',
80: 'Permanent Water Bodies',
90: 'Herbaceous Wetland',
100: 'Moss and Lichen',
111: 'Closed Evergreen Needle Leaf Forest',
112: 'Closed Evergreen Broad Leaf Forest',
113: 'Closed Deciduous Needle Leaft Forest',
114: 'Closed Deciduous Broad Leaf Forest',
115: 'Closed Mixed Forest',
116: 'Other Closed Forest',
121: 'Open Evergreen Needle Leaf Forest',
122: 'Open Evergreen Broad Leaf Forest',
123: 'Open Deciduous Needle Leaft Forest',
124: 'Open Deciduous Broad Leaf Forest',
125: 'Open Mixed Forest',
126: 'Other Open Forest',
200: 'Oceans, Seas'}

In [49]:
def rescale_image(raster):
  raster = np.nan_to_num(raster)
  max_val = np.nanmax(raster)
  mid_val = max_val/2
  rescaled = np.nan_to_num((raster-mid_val)/(mid_val))
  return np.clip(rescaled, -1, 1)

In [50]:
def rio_to_channels_last(raster):
  return raster.transpose((1, 2, 0))

In [51]:
def get_array(path):
  _r = tifffile.imread(path)
  #arr = rio_to_channels_last(_r)
  return _r

In [52]:
def get_image_paths(top_level_path):
  ecoregion_folders = glob.glob(top_level_path+'/*')
  img_paths = []
  for ec_dir in ecoregion_folders:
    img_paths += glob.glob(ec_dir+'*/*.tif')
  return img_paths


In [53]:
ecoregion_paths = 'latamSatData/DownloadedDataset/*'


In [54]:
from PIL import Image


In [63]:
img_paths = get_image_paths(ecoregion_paths)
for idx, p in enumerate(img_paths):
    arr = get_array(p)
    rgb = multispectral_to_rgb(arr)
    rgb = rgb.astype(np.uint8)

    split_string = p.split('/')
    ecoregionName = split_string[2]
    split_string[3] = classDictionary[int(split_string[3])]
    split_string[1] = "datasetRGB_rescaled"
    split_string[-1] = ecoregionName+ '_' + split_string[-1][:-4] + '.png'
    split_string.insert(2, 'all')
    split_string.pop(3)

    filePath = os.path.join('/'.join(split_string))
    dirPath = os.path.join('/'.join(split_string[:-1])+'/')
    if not os.path.isdir(dirPath):
        os.makedirs(dirPath)
    
    image = Image.fromarray(np.squeeze(rgb), mode="RGB")
    image.save(filePath)


    if  idx % 10000 == 0:
        print(idx, 'Done')

0 Done
10000 Done


/var/folders/p4/txj2zhrx5vx9ynlpynxc19980000gn/T/ipykernel_19417/3141652692.py:12: RuntimeWarning: invalid value encountered in divide
  rgb_raster = 255 * (rgb_raster - min)/(max-min)
/var/folders/p4/txj2zhrx5vx9ynlpynxc19980000gn/T/ipykernel_19417/3141652692.py:12: RuntimeWarning: divide by zero encountered in divide
  rgb_raster = 255 * (rgb_raster - min)/(max-min)


20000 Done
30000 Done
40000 Done
50000 Done
60000 Done
70000 Done
80000 Done
90000 Done
100000 Done
110000 Done
120000 Done
130000 Done
140000 Done
150000 Done
160000 Done
170000 Done


/var/folders/p4/txj2zhrx5vx9ynlpynxc19980000gn/T/ipykernel_19417/3141652692.py:9: RuntimeWarning: divide by zero encountered in log
  rgb_raster = np.log(rgb_raster)
/Users/joshredmond/miniconda3/envs/tensorflowMac/lib/python3.10/site-packages/numpy/lib/function_base.py:4573: RuntimeWarning: invalid value encountered in subtract
  diff_b_a = subtract(b, a)


180000 Done
190000 Done
200000 Done
210000 Done
220000 Done
230000 Done
240000 Done
250000 Done
260000 Done
270000 Done
280000 Done
290000 Done
300000 Done
310000 Done


In [58]:
filePath

'latamSatData/datasetRGB_rescaled/all/Urban Areas/135_tile_255361.png'

In [59]:
dirPath

'latamSatData/datasetRGB_rescaled/all/Urban Areas/'

In [ ]:
dirPath

In [ ]:
def prepare_for_training(ds, cache=True, batch_size=32, shuffle_buffer_size=1000):

  # shuffle the dataset
  #ds = ds.shuffle(buffer_size=shuffle_buffer_size)
  # Repeat forever
  #ds = ds.repeat()
  # split to batches
  ds = ds.batch(batch_size)
  # `prefetch` lets the dataset fetch batches in the background while the model
  # is training.
  ds = ds.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)
  return ds

In [ ]:
ecoregion_paths = 'latamSatData/DownloadedDataset/*'


In [ ]:

dataset = latamSatGenerator(ecoregion_paths, batch_size=1)
#train_img_generator = dataset.random_image_generator(normalise=True, rgb=True, split=[0,60])
#test_img_generator = dataset.random_image_generator(normalise=True, rgb=True, split=[61,90])
#val_img_generator = dataset.random_image_generator(normalise=True, rgb=True,split=[91,100])
train_img_generator = dataset.make_tf_dataset(normalise=True, rgb=False, supervised=True, split=[0,60])
test_img_generator = dataset.make_tf_dataset(normalise=True, rgb=False, split=[61,70])
val_img_generator = dataset.make_tf_dataset(normalise=True, rgb=False, split=[71,99])

In [ ]:
whole_ds_gen = dataset.random_image_generator(supervised=True, rgb=False, normalise=True, one_hot=False)

In [ ]:
ds_batch_size = 32

In [ ]:
def prepare_for_training(dataset, batch_size=32, cache='CachedDataset.cache'):
    dataset = dataset.repeat()

    dataset = dataset.batch(batch_size, drop_remainder=True)


    #dataset = dataset.shuffle(64)

    #dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

    return dataset

In [ ]:
val_img_generator = prepare_for_training(val_img_generator, cache='valCahce.cache')


In [ ]:
train_img_generator = prepare_for_training(train_img_generator, cache='trainCache.cache')


In [ ]:
test_img_generator = prepare_for_training(test_img_generator, cache='testCache.cache')

testreturn = next(val_img_generator.as_numpy_iterator())
testreturn[1].shape

In [ ]:
num_channels = testreturn[0].shape[-1]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
num_channels

In [ ]:
img_path_len = len(dataset.img_paths)
img_path_pct = np.floor(img_path_len/100)


In [ ]:
num_epochs = 20

In [ ]:
steps_epoch = (img_path_pct*60 // ds_batch_size) - 1 

In [ ]:
steps_epoch

In [ ]:
steps_validation = (img_path_pct*10 // ds_batch_size) -1

In [ ]:
steps_validation

In [ ]:
#train_ds = prepare_for_training(img_generator, batch_size=64)

In [ ]:
def make_model(input_shape, num_classes):
    inputs = tf.keras.Input(shape=input_shape)

    # Entry block
    #x = layers.Rescaling(1.0 / 255)(inputs)
    x = tf.keras.layers.Conv2D(128, 3, strides=2, padding="same")(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    previous_block_activation = x  # Set aside residual

    for size in [256, 512, 768]:
        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(size, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.Activation("relu")(x)
        x = tf.keras.layers.SeparableConv2D(size, 3, padding="same")(x)
        x = tf.keras.layers.BatchNormalization()(x)

        x = tf.keras.layers.MaxPooling2D(3, strides=2, padding="same")(x)

        # Project residual
        residual = tf.keras.layers.Conv2D(size, 1, strides=2, padding="same")(
            previous_block_activation
        )
        x = tf.keras.layers.add([x, residual])  # Add back residual
        previous_block_activation = x  # Set aside next residual

    x = tf.keras.layers.SeparableConv2D(1024, 3, padding="same")(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation("relu")(x)

    x = tf.keras.layers.GlobalAveragePooling2D()(x)
    if num_classes == 2:
        activation = "sigmoid"
        units = 1
    else:
        activation = "softmax"
        units = num_classes

    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(units, activation=activation)(x)
    return tf.keras.Model(inputs, outputs)


m = make_model(input_shape=(64,64,13), num_classes=19)
tf.keras.utils.plot_model(m, show_shapes=True)


In [ ]:

m.compile(
    loss="categorical_crossentropy",
    optimizer="adam",
    metrics=["accuracy"],
    run_eagerly=False
)

In [ ]:
import os

In [ ]:
model_name = "satellite-classification_xception"
model_path = os.path.join( model_name + ".h5")
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(model_path, save_best_only=True, verbose=1)

In [ ]:
try:
    m.load_weights(model_path)
except:
    pass

In [ ]:

def recursive_train():
    try:
        m_history = m.fit(
        train_img_generator,
        validation_data = test_img_generator,
        verbose=1, epochs=num_epochs,
        steps_per_epoch=steps_epoch,
        validation_steps = 25,
    
        callbacks=[model_checkpoint]
        )
    except:
        #print('epoch', epochs)
        pass
        

for i in range(1):
    print('starting epoch', i)
    #recursive_train()


In [ ]:
m.load_weights(model_path)
featureExtraction = tf.keras.Model(inputs=m.input,
                                 outputs=m.layers[-2].output)


In [ ]:
from cleanlab import Datalab


In [ ]:
feats_2 = []
classifications_2 = []
imgs_2 = []
pred_probs = []
for i in range(100001, img_path_len):
    img, probs = next(whole_ds_gen)
    img = np.expand_dims(img, 0)
    _f = featureExtraction.predict(img, verbose=0)
    pred = m.predict(img, verbose=0)
    feats_2.append(_f)
    imgs_2.append(img)
    pred_probs.append(pred)
    
    classifications_2.append(probs)
    if i > 0 and i % 1000 == 0:
        print(i, 'done')
    if i > 0 and i % 10000 == 0:
        data_to_clean = {'Images':np.squeeze(np.array(imgs_2)), 'Labels':np.array(classifications_2)}
        lab = Datalab(data=data_to_clean, label_name="Labels", image_key="Images")
        lab.find_issues(pred_probs=np.squeeze(np.array(pred_probs)), features=np.squeeze(np.array(feats_2)))
        label_issues = lab.get_issues("label")
        label_issues.to_csv(f"{i}_issues.csv")
        del(feats_2)
        del(classifications_2)
        del(imgs_2)
        del(pred_probs)
        feats_2 = []
        classifications_2 = []
        imgs_2 = []
        pred_probs = []
        

data_to_clean = {'Images':np.squeeze(np.array(imgs_2)), 'Labels':np.array(classifications_2)}
lab = Datalab(data=data_to_clean, label_name="Labels", image_key="Images")
lab.find_issues(pred_probs=np.squeeze(np.array(pred_probs)), features=np.squeeze(np.array(feats_2)))
label_issues = lab.get_issues("label")
label_issues.to_csv(f"{i}_issues.csv")



In [ ]:
break

In [ ]:
lab
#label_issues_df = label_issues.query("is_label_issue").sort_values("label_score")


In [ ]:
break

In [ ]:
feats_full = feats+ feats_2
classifications_full = classifications + classifications_2

import pickle
with open('featuresPickle.pickle', 'wb') as wf:
    pickle.dump(feats_full, wf)

with open('classesPickle.pickle', 'wb') as wc:
    pickle.dump(classifications_full, wc)

In [ ]:
print('Done!')